<a href="https://colab.research.google.com/github/mattshu0410/medical-reasoning-interp/blob/main/notebooks/BMJ_AI_CoT_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Dataset & Libraries

In [1]:
!pip install pydantic-ai
!pip install "pydantic-ai-slim[bedrock]"
!pip install "pydantic-ai-slim[anthropic]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.2/346.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 102.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.1/228.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!git clone https://github.com/kevinwu23/Stanford-MedCaseReasoning.git

Cloning into 'Stanford-MedCaseReasoning'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 37 (delta 17), reused 13 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 18.16 KiB | 2.59 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [3]:
!pip install -r /content/Stanford-MedCaseReasoning/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 30.9 MB/s eta 0:00:00


In [4]:
import asyncio
import pandas as pd
from datasets import load_dataset
import re
from pydantic import BaseModel, Field
from pydantic_ai import Agent, PromptedOutput
from pydantic_ai.models.bedrock import BedrockConverseModel, BedrockModelSettings
from pydantic_ai.models.anthropic import AnthropicModel, AnthropicModelSettings
from pydantic_ai.providers.anthropic import AnthropicProvider
from pydantic_ai.models.groq import GroqModel, GroqModelSettings
from pydantic_ai.providers.groq import GroqProvider
from pydantic_ai.settings import ModelSettings
from typing import List, Literal, Optional
from tqdm.asyncio import tqdm
import numpy as np

In [ ]:
ds = load_dataset("zou-lab/MedCaseReasoning", "default")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Output from Thao's DeepSeek

In [ ]:
# Replace 'path/to/your/file.csv' with the actual path to your file in Google Drive
try:
  df = pd.read_csv('/content/drive/MyDrive/Research/BMJ AI Paper/MedCaseReasoning DeepSeek 100 traces 100 cases.csv')
  display(df.head())
except FileNotFoundError:
  print("Error: File not found. Please check the path to your file.")

,pmcid,repeat_index,case_prompt,true_diagnosis,predicted_diagnosis,is_correct,reasoning_trace,verification_response
0,PMC4722580,95,A 19-year-old male presented with pain in the ...,cleidocranial dysplasia,Cleidocranial dysplasia,True,"First, I need to read the case presentation ca...",y
1,PMC4722580,91,A 19-year-old male presented with pain in the ...,cleidocranial dysplasia,Cleidocranial Dysplasia,True,"First, I need to read the case presentation ca...",y
2,PMC4722580,7,A 19-year-old male presented with pain in the ...,cleidocranial dysplasia,Cleidocranial dysplasia,True,"First, I need to read the case presentation ca...",y
3,PMC4722580,26,A 19-year-old male presented with pain in the ...,cleidocranial dysplasia,Cleidocranial dysplasia,True,"First, I need to read the case presentation ca...",y
4,PMC4722580,32,A 19-year-old male presented with pain in the ...,cleidocranial dysplasia,cleidocranial dysplasia,True,"First, I need to read the case presentation ca...",y


In [ ]:
ds["train"].column_names
medcase_df = ds["train"].to_pandas()
medcase_df

,Unnamed: 0,pmcid,title,journal,article_link,publication_date,text,case_prompt,diagnostic_reasoning,final_diagnosis
0,0,PMC4863315,A diffuse traumatic neuroma in the palate: a c...,Journal of Medical Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,2016-05-11,Background A traumatic neuroma is a hyperplast...,A 30-year-old man presented with several month...,1. Infection or inflammation was considered gi...,Traumatic neuroma
1,1,PMC5728002,Anterior Elbow Pain Caused by Compression of t...,Journal of Orthopaedic Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,2017,Learning Points for this Article: Compression ...,A 27-year-old left-hand–dominant man presented...,1. Biceps tendinopathy — “provocative testing ...,Lateral antebrachial cutaneous nerve compression
2,3,PMC4217568,Endosalpingiosis in Postmenopausal Elderly Women,Journal of Menopausal Medicine,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,2014-04-28,Introduction In gynecology endosalpingiosis is...,"A 75-year-old woman, gravida 2, postmenopausal...",1. Mucinous cystadenoma was the preoperative c...,Endosalpingiosis
3,4,PMC11003597,An Uncommon Complication of a Common Tropical ...,Indian Journal of Nephrology,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,2024-03-30,"Introduction Hemophagocytic syndrome, also cal...","A 27-year-old man, 6 months after deceased-don...",1. Immunosuppressant toxicity — “The most comm...,Hemophagocytic lymphohistiocytosis
4,5,PMC7042146,What is the cause of hypotension? A rare compl...,European Heart Journal: Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,2019-10-22,Learning points Differential diagnosis of hypo...,A 75-year-old woman with treated hypertension ...,1. Pericardial tamponade excluded — “This show...,Intramural haematoma
...,...,...,...,...,...,...,...,...,...,...
13087,14484,PMC6848952,Relapsing polychondritis with large airway inv...,Respirology Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,2019-11-12,Introduction Relapsing polychondritis is a rar...,A 57-year-old woman with a 20-year history of ...,1. Tracheobronchomalacia from relapsing polych...,relapsing polychondritis
13088,14485,PMC10276980,Idiopathic pneumoperitoneum in a single case: ...,Journal of Surgical Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,2023-06-17,INTRODUCTION Pneumoperitoneum is known to affe...,A 34-year-old man with type II diabetes mellit...,1. Gastric or duodenal ulcer perforation was c...,Idiopathic pneumoperitoneum
13089,14486,PMC2518156,Challenges in the prenatal and post-natal diag...,Journal of Medical Case Reports,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2...,2008-08-01,Introduction Cystic hygromas are slow-growing ...,A 27-year-old primigravida at 38 weeks’ gestat...,1. Congenital diaphragmatic hernia was conside...,cystic hygroma
13090,14487,PMC7641677,Marchiafava–Bignami Disease Associated with Sp...,Case Reports in Neurological Medicine,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,2020-10-28,1. Introduction Marchiafava–Bignami Disease (M...,A 53-year-old man was admitted with a two‐mont...,1. Aggressive multiple sclerosis was considere...,Marchiafava–Bignami disease


# Get Reasoning Trace

## Generic Helper Functions

In [ ]:
class VerificationOutput(BaseModel):
  """
  Structured output for diagnosis verification.
  """
  is_correct: Literal["y", "n"] = Field(
      description="Whether the predicted diagnosis matches the true diagnosis (y or n)."
  )
  explanation: str = Field(
      default="",
      description="Brief explanation of the verification decision."
  )

In [ ]:
def create_diagnosis_agent(
    model,
    ) -> Agent:
    """
    Create an agent for medical diagnosis using native text output.

    Args:
        model: Configured Bedrock model
    """

    instructions = """
    You are a medical expert. Read case presentations and provide diagnoses.
    Follow this exact output format:
    <answer>
    ...the name of the disease/entity...
    </answer>
    """
    agent = Agent(
        model,
        output_type=str,
        instructions=instructions,
    )
    return agent

def create_verification_agent(
    model,
) -> Agent:
    """
    Create an agent for verifying diagnosis correctness.
    Uses structured output since verification doesn't need chain of thought.

    Args:
        model: Configured Bedrock model
    """
    instructions = """
    You are evaluating whether a predicted diagnosis matches the true diagnosis.
    Consider semantic equivalence - different phrasings of the same condition should be considered correct.
    For example:
    - "Myocardial infarction" and "Heart attack" are the same
    - "Type 2 diabetes mellitus" and "Diabetes type 2" are the same
    - "Acute appendicitis" and "Appendicitis" are the same
    Be strict but fair in your evaluation.
    """

    agent = Agent(
        model,
        output_type=PromptedOutput(
            VerificationOutput,
            description="Diagnosis verification result"
        ),
        instructions=instructions,
    )
    return agent

In [ ]:
async def run_single_diagnosis(
    agent: Agent,
    case_prompt: str,
    pmcid: str,
    repetition_num: int
  ) -> dict:
    """
    Run a single diagnosis attempt.

    Args:
        agent: The diagnosis agent
        case_prompt: Prompt for the case
        pmcid: PMCID of the case
        repetition_num: Repetition number

    Returns:
        Dictionary with result data
    """
    # Format the prompt with the case
    prompt = f"""
    Read the following case presentation and give the most likely diagnosis.
    You must think very carefully, considering the information available to you.
    Then, output the final diagnosis (just the name of the disease/entity) within the tags <answer> ... </answer>.
    ---------------------------------------
    CASE PRESENTATION
    ---------------------------------------
    {case_prompt}
    ---------------------------------------
    OUTPUT TEMPLATE
    ---------------------------------------
    <answer>
    ...the name of the disease/entity...
    </answer>"""

    try:
      result = await agent.run(prompt)
      model_response = result.all_messages()[-1]
      reasoning = model_response.thinking if hasattr(model_response, 'thinking') else None
      text_output = model_response.text if hasattr(model_response, 'text') else None
      if text_output:
        answer_match = re.search(r'<answer>(.*?)</answer>', text_output, re.DOTALL | re.IGNORECASE)
        diagnosis = answer_match.group(1).strip() if answer_match else None
      else:
        diagnosis = None

      return {
          'pmcid': pmcid,
          'repetition': repetition_num,
          'reasoning_trace': reasoning,
          'predicted_diagnosis': diagnosis,
          'raw_response': model_response,
          'parsing_successful': reasoning is not None and diagnosis is not None,
          'success': True,
          'error': None
      }
    except Exception as e:
      return {
          'pmcid': pmcid,
          'repetition': repetition_num,
          'reasoning_trace': None,
          'predicted_diagnosis': None,
          'raw_response': None,
          'parsing_successful': False,
          'success': False,
          'error': str(e)
      }


async def run_case_with_repetitions(
    agent: Agent,
    row: pd.Series,
    n_repetitions: int = 100,
    max_concurrent: int = 10
) -> List[dict]:
    """
    Run a single case n times with concurrency control.

    Args:
        agent: The diagnosis agent
        row: DataFrame row with case data
        n_repetitions: Number of times to repeat (default 100)
        max_concurrent: Maximum concurrent API calls

    Returns:
        List of result dictionaries
    """
    semaphore = asyncio.Semaphore(max_concurrent)

    async def run_with_semaphore(rep_num):
        async with semaphore:
            return await run_single_diagnosis(
                agent,
                row['case_prompt'],
                row['pmcid'],
                rep_num
            )

    tasks = [run_with_semaphore(i) for i in range(n_repetitions)]
    results = await asyncio.gather(*tasks)

    # Add true diagnosis to each result
    for result in results:
        result['true_diagnosis'] = row['final_diagnosis']
        result['case_prompt'] = row['case_prompt']

    return results

In [ ]:
async def verify_diagnosis(
    agent: Agent,
    predicted_diagnosis: str,
    true_diagnosis: str
) -> dict:
    """
    Verify if a predicted diagnosis matches the true diagnosis.

    Returns:
        Dictionary with verification results
    """
    prompt = f"""Is the predicted diagnosis correct?

    Predicted diagnosis: {predicted_diagnosis}
    True diagnosis: {true_diagnosis}

    Answer y (correct) or n (incorrect)."""

    try:
        result = await agent.run(prompt)
        output = result.output

        return {
            'is_correct': output.is_correct,
            'explanation': output.explanation,
            'success': True,
            'error': None
        }
    except Exception as e:
        return {
            'is_correct': None,
            'explanation': None,
            'success': False,
            'error': str(e)
        }


async def verify_results(
    agent: Agent,
    results_df: pd.DataFrame,
    max_concurrent: int = 20
) -> pd.DataFrame:
    """
    Verify all diagnosis results.

    Args:
        verification_agent: The verification agent
        results_df: DataFrame with diagnosis results
        max_concurrent: Maximum concurrent verification calls

    Returns:
        DataFrame with verification results added
    """
    semaphore = asyncio.Semaphore(max_concurrent)

    async def verify_with_semaphore(row):
        async with semaphore:
            if pd.isna(row['predicted_diagnosis']) or row['predicted_diagnosis'] is None:
                return None
            return await verify_diagnosis(
                agent,
                row['predicted_diagnosis'],
                row['true_diagnosis']
            )

    print("Running verification...")
    tasks = []
    for idx, row in results_df.iterrows():
        tasks.append(verify_with_semaphore(row))

    verification_results = []
    for task in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Verifying"):
        result = await task
        verification_results.append(result)

    # Add verification columns
    results_df['is_correct'] = [
        r['is_correct'] if r and r['success'] else None
        for r in verification_results
    ]
    results_df['verification_explanation'] = [
        r['explanation'] if r and r['success'] else None
        for r in verification_results
    ]

    return results_df


## Set Preferences

You need to set your API keys in environment variables in Colab. I had to write explicitly because implicit wasn't working. Don't judge.

In [ ]:
from google.colab import userdata
ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

# Number of repetitions per case (default 100)
n_repetitions: int = 1
# Maximum concurrent API calls (if you get errors regarding rate limits drop this)
max_concurrent: int = 1
output_path: str = "results_df.parquet"


### Bedrock

Warning this is a pain in the arse to get set up. Use as a last resort.
https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html#API_runtime_Converse_RequestSyntax
https://docs.aws.amazon.com/bedrock/latest/userguide/kb-test-configure-reasoning.html

In [ ]:

# Model Settings

# diagnosis_model = BedrockConverseModel(
#     model_name="us.anthropic.claude-sonnet-4-20250514-v1:0",
#     settings=BedrockModelSettings(
#         bedrock_additional_model_requests_fields={
#             "inferenceConfig": {
#                 "temperature": 0.8,
#                 "topP": 0.95
#                 },
#             'thinking': {
#               'type': 'enabled',
#               'budget_tokens': 2048,
#               },
#             }
#     )
# )

# verification_model = BedrockConverseModel(
#     model_name="us.anthropic.claude-sonnet-4-20250514-v1:0",
#     settings=BedrockModelSettings(
#         bedrock_additional_model_requests_fields={
#             "inferenceConfig": {
#                 "temperature": 0.8,
#                 "topP": 0.95
#                 },
#         }
#     )
# )




### Claude or Opus

In [ ]:
# Note that you can't set temperature to anything other than 1 with thinking enabled
diagnosis_model = AnthropicModel(
    model_name = 'claude-sonnet-4-0', # Can swap out for claude-3-5-sonnet-latest
    settings = AnthropicModelSettings(
        anthropic_thinking={'type': 'enabled', 'budget_tokens': 1024},
    ),
    provider = AnthropicProvider(api_key = ANTHROPIC_API_KEY)
)

verification_model = AnthropicModel(
    model_name = 'claude-3-5-sonnet-latest',
    settings = AnthropicModelSettings(
        temperature = 0.8,
        top_p = 0.95
    ),
    provider = AnthropicProvider(api_key = ANTHROPIC_API_KEY)
)

### Qwen3 & GPT OSS


In [ ]:
diagnosis_model = GroqModel(
    model_name = 'qwen/qwen3-32b', # Can swap out with openai/gpt-oss-120b
    settings = GroqModelSettings(
        groq_reasoning_format='parsed',
    ),
    provider=GroqProvider(api_key=GROQ_API_KEY)
)

# We can still use Claude just for verification
verification_model = AnthropicModel(
    model_name = 'claude-3-5-sonnet-latest',
    settings = AnthropicModelSettings(
        temperature = 0.8,
        top_p = 0.95
    ),
    provider = AnthropicProvider(api_key = ANTHROPIC_API_KEY)
)

## Run Model

In [ ]:

# Create agents
diagnosis_agent = create_diagnosis_agent(diagnosis_model)
verification_agent = create_verification_agent(verification_model)

In [ ]:
medcase_df = medcase_df[1:5]

# Run diagnosis for all cases
all_results = []


print(f"Running diagnosis for {len(medcase_df)} cases with {n_repetitions} repetitions each...")
for idx, row in tqdm(medcase_df.iterrows(), total=len(medcase_df), desc="Cases"):
    case_results = await run_case_with_repetitions(
        diagnosis_agent,
        row,
        n_repetitions,
        max_concurrent
    )
    all_results.extend(case_results)

# Create results DataFrame
results_df = pd.DataFrame(all_results)

# Run verification
results_df = await verify_results(
    verification_agent,
    results_df,
    max_concurrent=max_concurrent * 2  # Verification is faster
)



Running diagnosis for 1 cases with 1 repetitions each...


Cases: 100%|██████████| 1/1 [00:10<00:00, 10.36s/it]


Running verification...


Verifying: 100%|██████████| 1/1 [00:02<00:00,  2.19s/it]


In [ ]:
results_df

,pmcid,repetition,reasoning_trace,predicted_diagnosis,raw_response,parsing_successful,success,error,true_diagnosis,case_prompt,is_correct,verification_explanation
0,PMC11003597,0,"Okay, let's see. The patient is a 27-year-old ...",Hemophagocytic Lymphohistiocytosis (HLH),"ModelResponse(parts=[ThinkingPart(content=""Oka...",True,True,None,Hemophagocytic lymphohistiocytosis,"A 27-year-old man, 6 months after deceased-don...",y,The predicted diagnosis 'Hemophagocytic Lympho...
1,PMC7042146,0,"Okay, let's see. The patient is a 75-year-old ...",Aortic dissection,"ModelResponse(parts=[ThinkingPart(content=""Oka...",True,True,None,Intramural haematoma,A 75-year-old woman with treated hypertension ...,n,While both conditions affect the aortic wall a...


In [ ]:
results_df.loc[1, :]['error']

"status_code: 400, model_name: qwen-qwq-32b, body: {'error': {'message': 'The model `qwen-qwq-32b` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}"